In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
from PIL import Image
import os

import ecoset
import categorization as cat
import utils

2023-05-15 00:03:55.622461: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 00:03:56.262216: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-15 00:03:56.262270: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-15 00:03:56.262276: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

# Dataset
100 images from Ecoset + CUB200.
* Superordinate: 50 vehicles and 50 animals
* Basic: 25 cars, 25 busses, 25 dogs, 25 birds
* Subordinate: 5 species of birds

Images preprocessed by centering the features and resizing to 224x224.

In [2]:
# Load csv
df = pd.read_csv("./deepCatsImages.csv")
df.head()


,path,name,cat1,cat2,cat3
0,./images/deepCats/animal/bird/CUB_005.Crested_...,Crested_Auklet_0021_794938.jpg,animal,bird,CUB_005.Crested_Auklet
1,./images/deepCats/animal/bird/CUB_005.Crested_...,Crested_Auklet_0028_794951.jpg,animal,bird,CUB_005.Crested_Auklet
2,./images/deepCats/animal/bird/CUB_005.Crested_...,Crested_Auklet_0036_794905.jpg,animal,bird,CUB_005.Crested_Auklet
3,./images/deepCats/animal/bird/CUB_005.Crested_...,Crested_Auklet_0011_794927.jpg,animal,bird,CUB_005.Crested_Auklet
4,./images/deepCats/animal/bird/CUB_005.Crested_...,Crested_Auklet_0006_1813.jpg,animal,bird,CUB_005.Crested_Auklet


In [3]:
imgFile = "./images/deepCatsImages.npy"

if not os.path.exists(imgFile):
    # Preallocate array for images
    images = np.zeros((df.shape[0], 224, 224, 3), dtype=np.float32)

    # Loop through images
    for i, row in df.iterrows():
        # Load image
        img = Image.open(row["path"])

        # Preprocess image
        img = ecoset.preprocess_alexnet(img)

        # Add image to array
        images[i] = img

    # Save images as a npy
    np.save(imgFile, images)
else:
    # Load images from npy
    print("Loading images from file...")
    images = np.load(imgFile)

print(images.shape)

Loading images from file...
(100, 224, 224, 3)


# Models and representations
Generate representations from AlexNet trained in Mehrer et al 2022. 
* 10 models are trained on Ecoset (basic-level categories)
* 10 models are trained on ImageNet (mixed levels of abstraction)

In [4]:
# Load example model
weightPath = f"./models/AlexNet/ecoset_training_seeds_01_to_10/training_seed_01/model.ckpt_epoch89"
model = ecoset.make_alex_net_v2(weights_path=weightPath)
model.summary()

2023-05-15 00:03:58.010722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 00:03:58.018652: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 00:03:58.019071: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 00:03:58.020373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Weights from ./models/AlexNet/ecoset_training_seeds_01_to_10/training_seed_01/model.ckpt_epoch89 loaded successfully.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 54, 54, 64)        23296     
                                                                 
 pool1 (MaxPooling2D)        (None, 26, 26, 64)        0         
                                                                 
 conv2 (Conv2D)              (None, 26, 26, 192)       307392    
                                                                 
 pool2 (MaxPooling2D)        (None, 12, 12, 192)       0         
                                                                 
 conv3 (Conv2D)              (None, 12, 12, 384)       663936    
         

In [5]:
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# Get ecoset representations
ecosetReps = np.zeros((len(seeds), images.shape[0], 4096 * 5 * 5), dtype=np.float32)
for i, seed in enumerate(seeds):
    repFile = f"./representations/ecoset/seed{seed:02}_reps.npy"
    
    if not os.path.exists(repFile):
        # Load model
        weightPath = f"./models/AlexNet/ecoset_training_seeds_01_to_10/training_seed_{seed:02}/model.ckpt_epoch89"
        model = ecoset.make_alex_net_v2(weights_path=weightPath)

        # Get activation from penultimate layer
        layer = model.get_layer("fc7")
        model = tf.keras.Model(model.input, layer.output)

        # Get representations
        reps = model.predict(images)
        # Flatten representation
        reps = reps.reshape(reps.shape[0], -1)

        # Save representations
        np.save(repFile, reps)
    else:
        print(f"Loading representations from file for seed {seed:02}...")
        reps = np.load(repFile)

    # Add rep to big representation array
    ecosetReps[i] = reps

ecosetReps.shape


Loading representations from file for seed 01...
Loading representations from file for seed 02...
Loading representations from file for seed 03...
Loading representations from file for seed 04...
Loading representations from file for seed 05...
Loading representations from file for seed 06...
Loading representations from file for seed 07...
Loading representations from file for seed 08...
Loading representations from file for seed 09...
Loading representations from file for seed 10...


(10, 100, 102400)

In [8]:
# Get imagenet representations
imagenetReps = np.zeros((len(seeds), images.shape[0], 4096 * 5 * 5), dtype=np.float32)
for i, seed in enumerate(seeds):
    repFile = f"./representations/imagenet/seed{seed:02}_reps.npy"
    
    if not os.path.exists(repFile):
        # Load model
        weightPath = f"./models/AlexNet/ILSVRC_training_seeds_01_to_10/training_seed_{seed:02}/model.ckpt_epoch89"
        model = ecoset.make_alex_net_v2(weights_path=weightPath,
                                        output_shape=1000)

        # Get activation from penultimate layer
        layer = model.get_layer("fc7")
        model = tf.keras.Model(model.input, layer.output)

        # Get representations
        reps = model.predict(images)
        # Flatten representation
        reps = reps.reshape(reps.shape[0], -1)

        # Save representations
        np.save(repFile, reps)
    else:
        print(f"Loading representations from file for seed {seed:02}...")
        reps = np.load(repFile)

    # Add rep to big representation array
    imagenetReps[i] = reps

imagenetReps.shape

Loading representations from file for seed 01...
Weights from ./models/AlexNet/ILSVRC_training_seeds_01_to_10/training_seed_02/model.ckpt_epoch89 loaded successfully.
4/4 [==============================] - 0s 52ms/step
Weights from ./models/AlexNet/ILSVRC_training_seeds_01_to_10/training_seed_03/model.ckpt_epoch89 loaded successfully.
4/4 [==============================] - 0s 51ms/step
Weights from ./models/AlexNet/ILSVRC_training_seeds_01_to_10/training_seed_04/model.ckpt_epoch89 loaded successfully.
4/4 [==============================] - 0s 49ms/step
Weights from ./models/AlexNet/ILSVRC_training_seeds_01_to_10/training_seed_05/model.ckpt_epoch89 loaded successfully.
4/4 [==============================] - 0s 48ms/step
Weights from ./models/AlexNet/ILSVRC_training_seeds_01_to_10/training_seed_06/model.ckpt_epoch89 loaded successfully.
4/4 [==============================] - 0s 49ms/step
Weights from ./models/AlexNet/ILSVRC_training_seeds_01_to_10/training_seed_07/model.ckpt_epoch89 load

2023-05-15 00:56:46.391997: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 100.02MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-15 00:56:46.447784: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 100.02MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-15 00:56:46.502862: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 100.02MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-15 00:56:46.562265: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocato

(10, 100, 102400)